In [1]:
from agential.cog.critic.prompts import CRITIC_CRITIQUE_INSTRUCTION_HOTPOTQA, CRITIC_INSTRUCTION_HOTPOTQA, HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC
from agential.cog.critic.strategies.qa import CriticQAStrategy
from agential.cog.fewshots.hotpotqa import HOTPOTQA_FEWSHOT_EXAMPLES_COT
from agential.llm.llm import LLM
from dotenv import load_dotenv
load_dotenv()
import os
llm = LLM("gpt-3.5-turbo")


In [2]:
from agential.cog.critic.prompts import CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_GSM8K, CRITIC_POT_INSTRUCTION_GSM8K, GSM8K_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL
from agential.cog.critic.strategies.math import CriticMathStrategy
from agential.cog.fewshots.gsm8k import GSM8K_FEWSHOT_EXAMPLES_POT


question = "Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with 4933828. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"

strat = CriticMathStrategy(llm=llm)

out = strat.generate(
    question=question,
    examples=GSM8K_FEWSHOT_EXAMPLES_POT,
    prompt=CRITIC_POT_INSTRUCTION_GSM8K,
    critique_examples=GSM8K_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL,
    critique_prompt=CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_GSM8K,
    additional_keys={},
    critique_additional_keys={},
    max_interactions=3,
    use_tool=False,
    reset=True,
)

<PROMPT AGENT=================================================>
# Write Python Code to solve the following questions. Store your result as a variable named 'answer'.

Question: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
# Python code, return answer
jason_lollipops_initial = 20
jason_lollipops_after = 12
denny_lollipops = jason_lollipops_initial - jason_lollipops_after
answer = denny_lollipops

---

Question: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
# Python code, return answer
trees_initial = 15
trees_after = 21
trees_added = trees_after - trees_initial
answer = trees_added

---

Question: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
# Python code, return answer
toys_initial = 5
mom_toys = 2
dad_toys = 2

In [4]:
out.additional_info

[CriticStepOutput(answer='eggs_laid_per_day = 16\neggs_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_sold_per_day = eggs_laid_per_day - eggs_for_breakfast - eggs_used_for_muffins\nprice_per_egg = 2\nearnings_per_day = eggs_sold_per_day * price_per_egg\nanswer = earnings_per_day', critique='There are multiple problems with the code provided:\n\n1. The variable `eggs_sold_per_day` is calculating the number of eggs sold at the market incorrectly. It should only consider the eggs left after breakfast and muffins, not subtract all the eggs used for breakfast and muffins.\n\n2. The variable `eggs_sold_per_day` is being used to calculate the amount earned per day, but it should be the number of eggs actually sold at the market that should be multiplied by the price per egg to get the earnings.\n\n3. The structure of the code is slightly confusing, with the variable names and calculations not aligning with the problem statement clearly.\n\n', external_tool_info={'execution_status': 

In [5]:
out.answer

'eggs_laid_per_day = 16\neggs_for_breakfast = 3\neggs_for_muffins = 6  # Assuming a reasonable number of eggs used for muffins\neggs_remaining = eggs_laid_per_day - eggs_for_breakfast - eggs_for_muffins\n\nprice_per_egg = 2\nearnings_per_day = eggs_remaining * price_per_egg\n\nanswer = earnings_per_day'

In [6]:
out

CriticOutput(answer='eggs_laid_per_day = 16\neggs_for_breakfast = 3\neggs_for_muffins = 6  # Assuming a reasonable number of eggs used for muffins\neggs_remaining = eggs_laid_per_day - eggs_for_breakfast - eggs_for_muffins\n\nprice_per_egg = 2\nearnings_per_day = eggs_remaining * price_per_egg\n\nanswer = earnings_per_day', total_prompt_tokens=10788, total_completion_tokens=822, total_tokens=11610, total_prompt_cost=0.0053939999999999995, total_completion_cost=0.0012330000000000002, total_cost=0.006626999999999999, total_prompt_time=10.707521438598633, total_time=11.989689588546753, additional_info=[CriticStepOutput(answer='eggs_laid_per_day = 16\neggs_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_sold_per_day = eggs_laid_per_day - eggs_for_breakfast - eggs_used_for_muffins\nprice_per_egg = 2\nearnings_per_day = eggs_sold_per_day * price_per_egg\nanswer = earnings_per_day', critique='There are multiple problems with the code provided:\n\n1. The variable `eggs_sold_per_day` i

In [7]:
from agential.llm.llm import MockLLM


responses = [
    'eggs_laid_per_day = 16\neggs_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_sold_per_day = eggs_laid_per_day - eggs_for_breakfast - eggs_used_for_muffins\nprice_per_egg = 2\nearnings_per_day = eggs_sold_per_day * price_per_egg\nanswer = earnings_per_day',
    "There are multiple problems with the code provided:\n\n1. The variable `eggs_sold_per_day` is calculating the number of eggs sold at the market incorrectly. It should only consider the eggs left after breakfast and muffins, not subtract all the eggs used for breakfast and muffins.\n\n2. The variable `eggs_sold_per_day` is being used to calculate the amount earned per day, but it should be the number of eggs actually sold at the market that should be multiplied by the price per egg to get the earnings.\n\n3. The structure of the code is slightly confusing, with the variable names and calculations not aligning with the problem statement clearly.\n\nHere's a corrected version of the code:\n\n```python\neggs_laid_per_day = 16\neggs_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_for_breakfast - eggs_used_for_muffins\nprice_per_egg = 2\nearnings_per_day = eggs_remaining * price_per_egg\nanswer = earnings_per_day\n```",
    'eggs_laid_per_day = 16\neggs_for_breakfast = 3\neggs_used_for_muffins = 4933828\n\neggs_remaining = eggs_laid_per_day - eggs_for_breakfast - eggs_used_for_muffins\nprice_per_egg = 2\nearnings_per_day = eggs_remaining * price_per_egg\n\nanswer = earnings_per_day\n```',
    "The total eggs used per day in this scenario exceeds the total eggs laid per day, which doesn't make sense. Additionally, the number of eggs used for muffins is unreasonably high at 4933828. \n\nHere's a better solution with reasonable values:\n\n```python\neggs_laid_per_day = 16\neggs_for_breakfast = 3\neggs_used_for_muffins = 5  # Assuming a more reasonable number\nprice_per_egg = 2\n\neggs_remaining = eggs_laid_per_day - eggs_for_breakfast - eggs_used_for_muffins\nearnings_per_day = eggs_remaining * price_per_egg\n\nanswer = earnings_per_day\n```",
    'eggs_laid_per_day = 16\neggs_for_breakfast = 3\neggs_for_muffins = 6  # Assuming a reasonable number of eggs used for muffins\neggs_remaining = eggs_laid_per_day - eggs_for_breakfast - eggs_for_muffins\n\nprice_per_egg = 2\nearnings_per_day = eggs_remaining * price_per_egg\n\nanswer = earnings_per_day\n```',
    "1. The total amount of earnings should be a positive number, 14 * 2 > 0, it's reasonable.\n\n2. Let's check the code:\n\n> eggs_laid_per_day = 16\n> eggs_for_breakfast = 3\n> eggs_for_muffins = 6\n\nThese variables correctly represent the number of eggs used for different purposes.\n\n> eggs_remaining = eggs_laid_per_day - eggs_for_breakfast - eggs_for_muffins\n\nThis calculates the number of eggs remaining for sale, which is correct.\n\n> price_per_egg = 2\n> earnings_per_day = eggs_remaining * price_per_egg\n\nThis calculates the earnings per day correctly by multiplying the remaining eggs with the price per egg.\n\n> answer = earnings_per_day\n\nThis assigns the correct earnings to the answer.\n\nOverall, the code correctly calculates the earnings Janet makes at the farmers' market every day. The calculations are done accurately based on the given information.",
    'eggs_laid_per_day = 16\neggs_for_breakfast = 3\neggs_for_muffins = 6  # Assuming a reasonable number of eggs used for muffins\neggs_remaining = eggs_laid_per_day - eggs_for_breakfast - eggs_for_muffins\n\nprice_per_egg = 2\nearnings_per_day = eggs_remaining * price_per_egg\n\nanswer = earnings_per_day',
]

llm = MockLLM("gpt-3.5-turbo", responses=responses)
strat = CriticMathStrategy(llm=llm, testing=True)

out = strat.generate(
    question=question,
    examples=GSM8K_FEWSHOT_EXAMPLES_POT,
    prompt=CRITIC_POT_INSTRUCTION_GSM8K,
    critique_examples=GSM8K_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL,
    critique_prompt=CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_GSM8K,
    additional_keys={},
    critique_additional_keys={},
    max_interactions=3,
    use_tool=False,
    reset=True,
)

<PROMPT AGENT=================================================>
# Write Python Code to solve the following questions. Store your result as a variable named 'answer'.

Question: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
# Python code, return answer
jason_lollipops_initial = 20
jason_lollipops_after = 12
denny_lollipops = jason_lollipops_initial - jason_lollipops_after
answer = denny_lollipops

---

Question: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
# Python code, return answer
trees_initial = 15
trees_after = 21
trees_added = trees_after - trees_initial
answer = trees_added

---

Question: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
# Python code, return answer
toys_initial = 5
mom_toys = 2
dad_toys = 2

In [8]:
out

CriticOutput(answer='eggs_laid_per_day = 16\neggs_for_breakfast = 3\neggs_for_muffins = 6  # Assuming a reasonable number of eggs used for muffins\neggs_remaining = eggs_laid_per_day - eggs_for_breakfast - eggs_for_muffins\n\nprice_per_egg = 2\nearnings_per_day = eggs_remaining * price_per_egg\n\nanswer = earnings_per_day', total_prompt_tokens=60, total_completion_tokens=120, total_tokens=180, total_prompt_cost=9e-05, total_completion_cost=0.00023999999999999998, total_cost=0.00033, total_prompt_time=3.0, total_time=0.5, additional_info=[CriticStepOutput(answer='eggs_laid_per_day = 16\neggs_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_sold_per_day = eggs_laid_per_day - eggs_for_breakfast - eggs_used_for_muffins\nprice_per_egg = 2\nearnings_per_day = eggs_sold_per_day * price_per_egg\nanswer = earnings_per_day', critique='There are multiple problems with the code provided:\n\n1. The variable `eggs_sold_per_day` is calculating the number of eggs sold at the market incorrectly